In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import dggs
from dggs.io import save_png, dump_text, load_polygons
from dggs import uitools
from dggs.datasets import get_path

dg = dggs.DGGS()

In [ ]:
poly, src_crs = load_polygons(get_path('act_aea.shp'))
poly = poly[0]
display(poly, src_crs)

In [ ]:
im = dggs.shape_to_mask(poly, src_crs, 12, align=1)

im

In [ ]:
ax = uitools.DgDraw(plt.figure(figsize=(6,6)))
ax.imshow(im)
ax.hide_axis()
ax.roi(im.roi, 'c-', linewidth=6, alpha=0.6)

In [ ]:
aa = dggs.mask_to_addresses(im, dg=dg)
print('\n'.join(aa[:5] + ['...'] + aa[-5:]))

In [ ]:
save_png('mm-mask-full-res.png', im.value, binary=True)
dump_text(aa, 'mm-addresses.txt')
dump_text(aa, 'mm-addresses.txt.gz')
dump_text(aa, 'mm-addresses.txt.xz')

!ls -lh mm-*

-------------------------------------------